In [9]:
!pip install tensorflow-text

     |████████████████████████████████| 4.9 MB 5.4 MB/s 
     |████████████████████████████████| 462 kB 47.9 MB/s 


In [1]:
import pandas as pd 
df = pd.read_csv("tweets.csv")
df.head()

,id,keyword,location,text,target
0,0,ablaze,NaN,"Communal violence in Bhainsa, Telangana. ""Ston...",1
1,1,ablaze,NaN,Telangana: Section 144 has been imposed in Bha...,1
2,2,ablaze,New York City,Arsonist sets cars ablaze at dealership https:...,1
3,3,ablaze,"Morgantown, WV",Arsonist sets cars ablaze at dealership https:...,1
4,4,ablaze,NaN,"""Lord Jesus, your love brings freedom and pard...",0


In [2]:
df = df.drop(['id','keyword','location'],axis = 1)

In [3]:
 df.head()

,text,target
0,"Communal violence in Bhainsa, Telangana. ""Ston...",1
1,Telangana: Section 144 has been imposed in Bha...,1
2,Arsonist sets cars ablaze at dealership https:...,1
3,Arsonist sets cars ablaze at dealership https:...,1
4,"""Lord Jesus, your love brings freedom and pard...",0


In [4]:
#Difference tensorflow version 1 & 2
#Tensorflow 1 - Graph processing
#Tensorflow 2 - Removed Graphs and Eager execution 

In [5]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [6]:
preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [8]:
df['target'].value_counts()

0    9256
1    2114
Name: target, dtype: int64

In [9]:
df_0 = df[df['target']==0]
df_1 = df[df['target']==1]
df_0_undersampled = df_0.sample(df_1.shape[0])

In [10]:
df_0_undersampled.shape

(2114, 2)

In [11]:
df = pd.concat([df_0_undersampled,df_1],axis = 0)

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_text, y_train, y_test = train_test_split(df['text'],df['target'],stratify = df['target'])

In [13]:
text_input = tf.keras.layers.Input(shape = (), dtype = tf.string,name='text-layer')
pre_processed_text = preprocess(text_input)
output = encoder(pre_processed_text)
d_layer = tf.keras.layers.Dropout(0.1,name = 'dropout_layer')(output['pooled_output'])
d_layer = tf.keras.layers.Dense(1,activation = 'sigmoid',name='output')(d_layer)
model = tf.keras.Model(inputs =[text_input],outputs = [d_layer])  

In [14]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text-layer (InputLayer)        [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['text-layer[0][0]']             
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [15]:
m = [
     tf.keras.metrics.BinaryAccuracy(name = 'accuracy'),
     tf.keras.metrics.Precision(name = 'precision'),
     tf.keras.metrics.Recall(name = 'recall')
]
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = m)

In [16]:
model.fit(X_train,y_train, epochs = 5)

Epoch 1/5
100/100 [==============================] - 105s 893ms/step - loss: 0.6645 - accuracy: 0.5935 - precision: 0.5955 - recall: 0.5823
Epoch 2/5
100/100 [==============================] - 91s 908ms/step - loss: 0.6072 - accuracy: 0.6755 - precision: 0.6735 - recall: 0.6808
Epoch 3/5
100/100 [==============================] - 81s 813ms/step - loss: 0.5819 - accuracy: 0.7089 - precision: 0.7006 - recall: 0.7293
Epoch 4/5
100/100 [==============================] - 82s 815ms/step - loss: 0.5721 - accuracy: 0.7140 - precision: 0.7129 - recall: 0.7161
Epoch 5/5
100/100 [==============================] - 81s 809ms/step - loss: 0.5570 - accuracy: 0.7241 - precision: 0.7264 - recall: 0.7186


In [17]:
model.evaluate(X_text, y_test)

34/34 [==============================] - 28s 797ms/step - loss: 0.5585 - accuracy: 0.7181 - precision: 0.6995 - recall: 0.7656


[0.5584824085235596,
 0.7180700302124023,
 0.6994818449020386,
 0.7655954360961914]

In [18]:
import numpy as np 
from sklearn.metrics import confusion_matrix, classification_report
y_predicted = model.predict(X_text)
y_predicted = y_predicted.flatten()
y_predicted = np.where(y_predicted > 0.5, 1,0)
matrix = confusion_matrix(y_test, y_predicted)
matrix

array([[354, 174],
       [124, 405]])